<b>
Final Exam: Model Deployment in the Cloud

Name: Diego Angelo G. Espiritu

Section: CPE32S1

Professor: Engr. Roman Richard

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import urllib.request
from io import BytesIO
from PIL import Image
import os
from tensorflow.keras.utils import image_dataset_from_directory

In [ ]:
!pip install keras

In [ ]:
from google.colab import files
files.upload()  # This will prompt you to upload the kaggle.json file


Saving kagglee.json to kagglee.json


{'kagglee.json': b'{"username":"andrejhonanot","key":"67cab683d244e37f6da7cd62838f9f94"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kagglee.json ~/.kaggle/
!chmod 600 ~/.kaggle/kagglee.json

In [ ]:
!kaggle datasets download -d mrtontrnok/5-vehichles-for-multicategory-classification

Dataset URL: https://www.kaggle.com/datasets/mrtontrnok/5-vehichles-for-multicategory-classification
License(s): CC0-1.0
100% 464M/465M [00:06<00:00, 85.1MB/s]
100% 465M/465M [00:06<00:00, 77.6MB/s]


In [ ]:
import zipfile
import os

# Create a directory to unzip the dataset
!mkdir -p /content/extracted

# Unzip the dataset
with zipfile.ZipFile('5-vehichles-for-multicategory-classification.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/extracted')

# Check the extracted files
os.listdir('/content/extracted')


['dataset']

In [ ]:
!pip install kaggle

In [ ]:
data_dir = '/kaggle/input/5-vehichles-for-multicategory-classification/dataset'
train_dir = '/content/extracted/dataset/train'
validation_dir = '/content/extracted/dataset/validation'
test_dir = '/content/extracted/dataset/test'

In [ ]:
# Set parameters
batch_size = 32
image_size = (192, 192)
num_of_classes = 5

learning_rate = 0.001
optimizer = Adam(learning_rate = learning_rate)
epoch_num = 15

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Add
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, GlobalAveragePooling2D, Rescaling,SeparableConv2D
from keras.optimizers import Adam,SGD


def create_model():
    inputs = tf.keras.Input(shape=(192, 192, 3))

    x = Conv2D(256, 3, strides=2, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [64,128,256]:
        x = Activation("relu")(x)
        x = SeparableConv2D(size, 3, padding="same")(x)
        x = BatchNormalization()(x)

        x = Activation("relu")(x)
        x = SeparableConv2D(size, 3, padding="same")(x)
        x = BatchNormalization()(x)

        x = MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = tf.keras.layers.Add()([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = SeparableConv2D(256, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = GlobalAveragePooling2D()(x)

    x = Dropout(0.5)(x)

    outputs = Dense(5, activation="softmax")(x)
    return tf.keras.Model(inputs, outputs)

In [ ]:
model = create_model()

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 192, 192, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 96, 96, 256)          7168      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 96, 96, 256)          1024      ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 96, 96, 256)          0         ['batch_normalization[0][0

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
seed = 43
shuffle = True


train_data_generator = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_data_generator = ImageDataGenerator(
    rescale=1./255  #
)

In [ ]:
# Training data generator
train_dataset = train_data_generator.flow_from_directory(
    train_dir,  # Path to the training data directory
    batch_size=batch_size,
    target_size=image_size,
    class_mode = 'categorical',
    shuffle=shuffle,
    seed=seed
)


validation_dataset = val_data_generator.flow_from_directory(
    validation_dir,
    batch_size=batch_size,
    target_size=image_size,
    class_mode = 'categorical',
    shuffle=shuffle,
    seed=seed
)


test_dataset = val_data_generator.flow_from_directory(
    test_dir,
    batch_size=batch_size,
    target_size=image_size,
    class_mode = 'categorical',
    shuffle=shuffle,
    seed=seed
)

Found 5418 images belonging to 5 classes.
Found 709 images belonging to 5 classes.
Found 708 images belonging to 5 classes.


In [ ]:
def scheduler(epoch, lr):
    if epoch == 15:
        return lr * 0.1
    elif epoch > 15:
        return lr * tf.math.exp(-0.1)
    else:
        return lr

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

callbacks = [
         EarlyStopping(patience=7, restore_best_weights=True),
        ModelCheckpoint("Best_Model.h5", save_best_only=True),
        tf.keras.callbacks.LearningRateScheduler(scheduler),
]

In [ ]:
history = model.fit(train_dataset,
                       epochs=epoch_num,
                       steps_per_epoch=train_dataset.samples // batch_size,
                       validation_data=validation_dataset,
                       validation_steps=validation_dataset.samples // batch_size,
                       callbacks = callbacks)

Epoch 1/15
169/169 [==============================] - 1284s 8s/step - loss: 1.2473 - accuracy: 0.4955 - val_loss: 1.7245 - val_accuracy: 0.2585 - lr: 0.0010
Epoch 2/15


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


169/169 [==============================] - 1331s 8s/step - loss: 0.9778 - accuracy: 0.6240 - val_loss: 2.2310 - val_accuracy: 0.2727 - lr: 0.0010
Epoch 3/15
169/169 [==============================] - 1298s 8s/step - loss: 0.8712 - accuracy: 0.6619 - val_loss: 1.5805 - val_accuracy: 0.3594 - lr: 0.0010
Epoch 4/15
169/169 [==============================] - 1295s 8s/step - loss: 0.8382 - accuracy: 0.6786 - val_loss: 0.8615 - val_accuracy: 0.6747 - lr: 0.0010
Epoch 5/15
169/169 [==============================] - 1259s 7s/step - loss: 0.7690 - accuracy: 0.7118 - val_loss: 2.0751 - val_accuracy: 0.5085 - lr: 0.0010
Epoch 6/15
169/169 [==============================] - 1298s 8s/step - loss: 0.7338 - accuracy: 0.7239 - val_loss: 1.4464 - val_accuracy: 0.5355 - lr: 0.0010
Epoch 7/15
169/169 [==============================] - 1301s 8s/step - loss: 0.6851 - accuracy: 0.7466 - val_loss: 1.7841 - val_accuracy: 0.5199 - lr: 0.0010
Epoch 8/15
169/169 [==============================] - 1296s 8s/step -

In [ ]:
Best_Model = tf.keras.models.load_model('Best_Model.h5')

OSError: No file or directory found at Best_Model.h5

In [ ]:
print(tf.keras.__version__)
print(keras.__version__)